In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, reset_replay, disable_replay

enable_replay(False)

set_defaults(
    cad_width=780, 
    height=540, 
)

print()
versions()

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay

jupyter_cadquery  3.1.0rc4
cad_viewer_widget 1.3.2
OCP               7.5.3


In [2]:
use_replay = True

if use_replay:
    enable_replay(False, False)
    reset_replay()
    show_object = replay
else:
    disable_replay()
    show_object = show


Enabling jupyter_cadquery replay


# Bar Mount

## Wall Bar Anchor

In [3]:
outer_height = 23
outer_depth = 20
inner_height = 22
inner_depth = 15

back_spacer = outer_depth - inner_depth
ground_spacer = outer_height - inner_height

wall_thickness = 2
trench_depth = 9
trench_gap = 4.5

grip_thickness = 1.6
grip_mount_height = 6
grip_grip = 0.5
grip_length = 21

l_length = 60 - 2 * grip_length

bottom_thickness = 3

bottom_bracket_height = 7.2
bottom_bracket_indention = 6

In [4]:
lp00 = (0.,0)
lp01 = (inner_height,0.)
lp1 = (inner_height,wall_thickness)
lp2 = (bottom_thickness,wall_thickness)
lp3 = (bottom_thickness,inner_depth)
lp4 = (0,inner_depth)

grip_sketch = (
    cq.Sketch()
    .segment(lp00, lp01)
    .segment(lp1)
    .segment((inner_height - trench_depth,wall_thickness))
    .segment((inner_height - trench_depth,wall_thickness + trench_gap + grip_grip))
    .segment((inner_height,wall_thickness + trench_gap))
    .segment((inner_height,wall_thickness + trench_gap + grip_thickness))
    .segment((inner_height - trench_depth,wall_thickness + trench_gap + grip_thickness + grip_grip))
    .segment((inner_height - trench_depth - wall_thickness,wall_thickness + trench_gap + grip_thickness))
    .segment((grip_mount_height,wall_thickness))
    .segment(lp2)
    .segment((bottom_thickness,inner_depth - wall_thickness))
    .segment((bottom_bracket_height,inner_depth - bottom_bracket_indention))
    .segment((bottom_bracket_height,inner_depth - bottom_bracket_indention + wall_thickness))
    .segment(lp3)
    .segment(lp4)
    .close()
    .assemble(tag='grip_face')
    .edges('%LINE',tag='grip_face')
    # .vertices('>X or >Y or <X or <Y')
    # .vertices()
    # .fillet(0.75)
    .reset()
)

show_object(grip_sketch)

Use the multi select box below to select one or more steps you want to examine


In [5]:
l_sketch = (
    cq.Sketch()
    .segment(lp00, lp01)
    .segment(lp1)
    .segment(lp2)
    .segment(lp3)
    .segment(lp4)
    .close()
    .assemble(tag='l_face')
    .edges('%LINE',tag='l_face')
    # .vertices()
    # .fillet(0.75)
    .reset()
)

show_object(l_sketch)

Use the multi select box below to select one or more steps you want to examine


In [6]:
spacer_sketch = (
    cq.Sketch()
    .segment(lp00, lp01)
    .segment((inner_height, -back_spacer))
    .segment((0, -back_spacer))
    .segment((-ground_spacer, 0))
    .segment((-ground_spacer, inner_depth))
    .segment(lp4)
    .close()
    .assemble(tag='spacer_face')
    .edges('%LINE',tag='spacer_face')
    .vertices('<Y or <X')
    .chamfer(1)
    .reset()
)

show_object(spacer_sketch)

Use the multi select box below to select one or more steps you want to examine


In [7]:
def make_grip_section():
    grip_section = (
        cq.Workplane('ZY')
        .placeSketch(grip_sketch)
        .extrude(grip_length / 2, both=True)
        .workplane()
        .placeSketch(spacer_sketch)
        .extrude(5 / 2, both=True, combine=True)
        .combine()
    )

    grip_section.vertices('>X').vertices('>Y').vertices('<Z').tag('left_point_front')
    grip_section.vertices('>X').vertices('<Y').vertices('<Z').tag('left_point')
    grip_section.vertices('>X').vertices('<Y').vertices('>Z').tag('left_point_up')
    grip_section.vertices('<X').vertices('>Y').vertices('<Z').tag('right_point_front')
    grip_section.vertices('<X').vertices('<Y').vertices('<Z').tag('right_point')
    grip_section.vertices('<X').vertices('<Y').vertices('>Z').tag('right_point_up')
    return grip_section
    
show_object(make_grip_section())

Use the multi select box below to select one or more steps you want to examine


In [8]:
def make_l_section(length):
    inner_bore_height = 14

    l_section = (
        cq.Workplane('ZY')
        .placeSketch(l_sketch)
        .extrude(length / 2, both=True)
    )
    l_section.vertices('>X').vertices('>Y').vertices('<Z').tag('left_point_front')
    l_section.vertices('>X').vertices('<Y').vertices('<Z').tag('left_point')
    l_section.vertices('>X').vertices('<Y').vertices('>Z').tag('left_point_up')
    l_section.vertices('<X').vertices('>Y').vertices('<Z').tag('right_point_front')
    l_section.vertices('<X').vertices('<Y').vertices('<Z').tag('right_point')
    l_section.vertices('<X').vertices('<Y').vertices('>Z').tag('right_point_up')
    return l_section

show_object(make_l_section(l_length))

Use the multi select box below to select one or more steps you want to examine


In [9]:
def make_l_csk_section(length):
    inner_bore_height = 14

    l_section = (
        cq.Workplane('ZY')
        .placeSketch(l_sketch)
        .extrude(length / 2, both=True)
        .faces(">Y[-2]").workplane()
        .center(0, inner_bore_height)
        .cskHole(4, 8, 100, depth=None)
    )
    l_section.vertices('>X').vertices('>Y').vertices('<Z').tag('left_point_front')
    l_section.vertices('>X').vertices('<Y').vertices('<Z').tag('left_point')
    l_section.vertices('>X').vertices('<Y').vertices('>Z').tag('left_point_up')
    l_section.vertices('<X').vertices('>Y').vertices('<Z').tag('right_point_front')
    l_section.vertices('<X').vertices('<Y').vertices('<Z').tag('right_point')
    l_section.vertices('<X').vertices('<Y').vertices('>Z').tag('right_point_up')
    return l_section

show_object(make_l_csk_section(l_length))

Use the multi select box below to select one or more steps you want to examine


In [31]:
def make_l_slot_section(length, slot):
    inner_bore_height = 14

    l_section = (
        cq.Workplane('ZY')
        .placeSketch(l_sketch)
        .extrude(length / 2, both=True)
        .faces(">Y[-2]").workplane()
        .center(0, inner_bore_height)
        .slot2D(slot, 4).tag('slot')
        .cutThruAll()
        .faces('<Y[1]')
        .edges('not (>X or <X or >Z or <Z)')
        .chamfer(1.6)
    )
    l_section.vertices('>X').vertices('>Y').vertices('<Z').tag('left_point_front')
    l_section.vertices('>X').vertices('<Y').vertices('<Z').tag('left_point')
    l_section.vertices('>X').vertices('<Y').vertices('>Z').tag('left_point_up')
    l_section.vertices('<X').vertices('>Y').vertices('<Z').tag('right_point_front')
    l_section.vertices('<X').vertices('<Y').vertices('<Z').tag('right_point')
    l_section.vertices('<X').vertices('<Y').vertices('>Z').tag('right_point_up')
    return l_section

show_object(make_l_slot_section(l_length, 10))

Use the multi select box below to select one or more steps you want to examine


## Assamble the parts

In [ ]:
disable_replay()
show_object = show

In [ ]:
def make_basic_mount():
    basic_mount = (
        cq.Assembly()
        .add(make_grip_section(), name="g0")
        .add(make_l_section(l_length), name="l0")
        .add(make_grip_section(), name="g1")
    )
    (
        basic_mount
        .constrain("g0?left_point", "Fixed")
        .constrain("g0?right_point_front", "l0?left_point_front", "Point")
        .constrain("g0?right_point", "l0?left_point", "Point")
        .constrain("g0?right_point_up", "l0?left_point_up", "Point")
        .constrain("l0?right_point_front", "g1?left_point_front", "Point")
        .constrain("l0?right_point", "g1?left_point", "Point")
        .constrain("l0?right_point_up", "g1?left_point_up", "Point")
    )
    basic_mount.solve()
    return basic_mount

show_object(make_basic_mount())

In [ ]:
def make_mount():
    mount = (
        cq.Assembly()
        .add(make_basic_mount(), name="m0")
        .add(make_basic_mount(), name="m1")
    )
    (
        mount
        # .constrain("m0@g0?left_point", "Fixed")
        # .constrain("m0@g0?right_plane", "m1@l0?left_plane", "PointInPlane")
        # .constrain("m0@g0?right_point", "m1@l0?left_point", "Point")
        # .constrain("m0@g0?right_point_up", "m1@l0?left_point_up", "Point")
    )
    # mount.solve()
    return mount
show_object(make_mount())

In [ ]:
class Wallmount_Constraints:
    def __init__(self):
        self.assembly = cq.Assembly()
        self.names = []

    def add_section(self, section, name: str):
        self.assembly.add(section, name=name)
        if len(self.names) == 0:
            (
                self.assembly
                .constrain(name + "?left_point_front", "Fixed")
                .constrain(name + "?left_point", "Fixed")
                .constrain(name + "?left_point_up", "Fixed")
            )
        else:
            last_name = self.names[-1]
            (
                self.assembly
                .constrain(last_name + "?right_point_front", name + "?left_point_front", "Point")
                .constrain(last_name + "?right_point", name + "?left_point", "Point")
                .constrain(last_name + "?right_point_up", name + "?left_point_up", "Point")
            )
            for _ in range(10):
                self.assembly = self.assembly.solve()
            # show_object(self.assembly)
        self.names.append(name)

    def add_l_section(self, name: str, length: float):
        self.add_section(make_l_section(length), name)
        return self

    def add_grip_section(self, name: str):
        self.add_section(make_grip_section(), name)
        return self
    
    def make(self):
        self.assembly.solve()
        return self.assembly

In [ ]:
mount = Wallmount_Constraints()

(
    mount
    .add_grip_section("g0")
    .add_l_section("l0", 40)
    .add_grip_section("g1")
    # .add_l_section("l1", 40)
    # .add_grip_section("g2")
    # .add_l_section("l2", 40)
    # .add_grip_section("g3")
    .make()
)

In [ ]:
class Wallmount_Singleton:
    def __init__(self):
        self.mount = None

    def add_tags(self):
        self.mount.vertices('>X').vertices('>Y').vertices('<Z').tag('left_point_front')
        self.mount.vertices('>X').vertices('<Y').vertices('<Z').tag('left_point')
        self.mount.vertices('>X').vertices('<Y').vertices('>Z').tag('left_point_up')
        self.mount.vertices('<X').vertices('>Y').vertices('<Z').tag('right_point_front')
        self.mount.vertices('<X').vertices('<Y').vertices('<Z').tag('right_point')
        self.mount.vertices('<X').vertices('<Y').vertices('>Z').tag('right_point_up')
        
    def add_section(self, section):

        if self.mount == None:
            self.mount = section
        else:
            assembly = cq.Assembly()

            name = "new"
            last_name = "old"

            assembly.add(self.mount, name=name)
            assembly.add(section, name=last_name)

            (
                assembly
                .constrain(name + "?left_point_front", "Fixed")
                .constrain(name + "?left_point", "Fixed")
                .constrain(name + "?left_point_up", "Fixed")
                .constrain(last_name + "?right_point_front", name + "?left_point_front", "Point")
                .constrain(last_name + "?right_point", name + "?left_point", "Point")
                .constrain(last_name + "?right_point_up", name + "?left_point_up", "Point")
            )

            for _ in range(2):
                assembly = assembly.solve()
            self.mount = cq.Workplane(assembly.toCompound())
            self.add_tags()
            # show_object(self.mount)

    def add_l_section(self, length: float):
        self.add_section(make_l_section(length))
        return self

    def add_grip_section(self):
        self.add_section(make_grip_section())
        return self
    
    def make(self):
        return self.mount

In [ ]:
mount = Wallmount_Singleton()

(
    mount
    .add_grip_section()
    .add_l_section(40)
    .add_grip_section()
    .add_l_section(40)
    .add_grip_section()
    .add_l_section(40)
    .add_grip_section()
    .add_l_section(40)
    .add_grip_section()
)

show_object(mount.make())